<a href="https://colab.research.google.com/github/charlie-aashutosh/charlie/blob/master/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are going to make a Hollywood movie recommender system in Python. Here we are using the IMDB-5000 Movie Dataset from Kaggle. You can download the dataset from this link :- https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
files=files.upload()

Saving movie_metadata.csv to movie_metadata.csv


In [82]:
data=pd.read_csv("movie_metadata.csv")

In [83]:
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [88]:
data.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [89]:
data.isnull().sum(axis=0)

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

In [90]:
#we will fill the nan values of the features we are going to consider for creating our Recommendation System
data['actor_1_name']=data['actor_1_name'].replace(np.nan, 'unknown')
data['actor_2_name']=data['actor_2_name'].replace(np.nan, 'unknown')
data['actor_3_name']=data['actor_3_name'].replace(np.nan, 'unknown')

In [91]:
data['director_name']=data['director_name'].replace(np.nan, 'unknown')

In the ‘genres’ column, replacing the ‘|’ with whitespace, so the genres would be considered different strings

In [92]:
data['genres']=data['genres'].str.replace('|', ' ')
data['genres'].head()


0    Action Adventure Fantasy Sci-Fi
1           Action Adventure Fantasy
2          Action Adventure Thriller
3                    Action Thriller
4                        Documentary
Name: genres, dtype: object

Now converting the ‘movie_title’ columns values to lowercase for searching simplicity.

In [93]:

data['movie_title']=data['movie_title'].str.lower()

In [94]:
data['movie_title'].head()

0                                              avatar 
1            pirates of the caribbean: at world's end 
2                                             spectre 
3                               the dark knight rises 
4    star wars: episode vii - the force awakens    ...
Name: movie_title, dtype: object

One more operation we need to perform on the title column. All the title strings has one special character in the end, which will cause problem while searching for the movie in dataset. Hence removing the last character from all the elements of ‘movie_title’.

In [95]:
data['movie_title']=data['movie_title'].str[:-1]

In [96]:
data['comb']=data['actor_1_name']+' '+data['actor_2_name']+' '+data['actor_3_name']+' '++data['genres']+' '+data['director_name']

In [97]:
data['comb'].head()

0    CCH Pounder Joel David Moore Wes Studi Action ...
1    Johnny Depp Orlando Bloom Jack Davenport Actio...
2    Christoph Waltz Rory Kinnear Stephanie Sigman ...
3    Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4    Doug Walker Rob Walker unknown Documentary Dou...
Name: comb, dtype: object

Now after getting all the features summed up in one column in text form. We can make a similarity matrix. For making a similarity matrix, first we have to create count matrix of features.

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

After importing the libraries, now let’s make count matrix of features and then similarity Score matrix. Similarity score matrix contains the cosine similarity of all the movies in the dataset. We can get the similarity score of two movies by there index in dataset.

In [99]:

#creating a count matrix
cv=CountVectorizer()
count_matrix=cv.fit_transform(data['comb'])
#computing the similarity score matrix
sim=cosine_similarity(count_matrix)

In [100]:
sim

array([[1.        , 0.24174689, 0.16116459, ..., 0.0805823 , 0.        ,
        0.        ],
       [0.24174689, 1.        , 0.18181818, ..., 0.        , 0.        ,
        0.        ],
       [0.16116459, 0.18181818, 1.        , ..., 0.09090909, 0.        ,
        0.        ],
       ...,
       [0.0805823 , 0.        , 0.09090909, ..., 1.        , 0.0836242 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0836242 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

Now, get the index of the movie in the dataset. Then fetch the row on the same index from the similarity score matrix, which has the similarity scores of all the movies to the movie user like.

In [105]:
#movie user like
m='The Dark Knight Rises'
m=m.lower()

In [106]:
m in data['movie_title'].unique()

True

In [107]:


#getting the index of the movie from the dataset
i=data.loc[data['movie_title'] == m].index[0]
print(i)

3


In the above step we also enumerated the row of similarity score, because for the most similar movies we have to sort this list in descending order, but we don’t want to lose the original index of similar movies, hence enumerating is important and sorting the similarity score list on the basis of similarity score is also requried

In [108]:

#getting the row related to the movie- from the similarity score matri.Also enumerating row so that we dont loose indexes afteer sorting in form of list

In [109]:
lst=list(enumerate(sim[i]))

Now we have the indexes of ten most similar movies to the movie user likes in the list. We just need to iterate through the list and store movie names on the indexes in a new list and print it.

In [110]:
lst=sorted(lst,key = lambda x:x[1], reverse=True)

In [111]:
lst=lst[1:11]

In [112]:
#finally defining a function which will take movie from user as input and print top 10 most relevant movies according to preference

def recommend(m):
  m=m.lower()
  #checking if given movie is in our dataset or not
  if m not in data['movie_title'].unique():
    print("This movie is not available at the moment. \nPlease choose another one")

  else:
    i=data.loc[data['movie_title'] == m].index[0]
    lst=list(enumerate(sim[i]))
    lst=sorted(lst,key=lambda x:x[1], reverse=True)
    lst=lst[1:11]
    l=[]

    for i in range(len(lst)):
      a=lst[i][0]
      l.append(data['movie_title'][a])
      return 1

    for i in range(len(l)):
      print(l[i])

       